In [1]:
library(lme4)
library(lmerTest)
library(stringr)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [4]:
data_profile <- read.csv('../../data/drug_effect_67_met_profile.public.txt', row.names = 1, header=TRUE, sep="\t")
data_profile <- as.data.frame(t(data_profile))
output_file <- 'drug_effect_67_met_profile.results.public.tsv'


In [5]:
head(data_profile)

,patient_dummyID,SEX,AGE,mtx,csdmard,tnfi,ntnfi,pred,100002254,100000257,⋯,100002911,1489,100020361,100001104,100000269,100020866,100001510,100001767,171,344
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
temp_0001,0,1,75,1,0,0,0,0,4.2646,1.5674,⋯,1.6174,5.8065,1.5960,2.3718,0.7689,4.2787,2.8148,1.5577,0.0243,1.3874
temp_0002,0,1,76,1,0,0,0,0,0.7423,1.8382,⋯,0.9193,0.5310,1.1545,1.4633,0.9511,4.7451,2.6375,0.8981,0.2159,1.3073
temp_0003,1,1,70,1,0,1,0,1,10.7037,0.8284,⋯,1.3136,7.5976,1.3427,1.1518,1.3642,1.7638,0.7221,3.1478,0.1789,1.3242
temp_0004,1,1,71,1,0,1,0,1,0.8509,0.7866,⋯,3.3806,0.4750,0.9460,0.8480,0.7717,1.4401,1.4132,0.5696,0.4390,1.1715
temp_0005,2,0,69,0,1,0,0,0,2.2357,1.1446,⋯,0.3923,2.7211,1.1114,0.8064,2.3825,0.9498,0.9553,1.2109,0.1584,1.0916
temp_0006,2,0,69,0,1,0,0,0,0.5838,0.9489,⋯,2.5219,0.4712,1.0083,0.8661,1.2101,0.9348,1.5982,0.3697,1.0086,1.5167


In [6]:
NUM_CHEM_ID <- ncol(data_profile)

In [7]:
if (file.exists(output_file)){
    file.remove(output_file)
}

cat("chemID\tMTX_LRM\tMTX_MIX1\tMTX_MIX2\tcsDMARD_LRM\tcsDMARD_MIX1\tcsDMARD_MIX2\tTNFi_LRM\tTNFi_MIX1\tTNFi_MIX2\tnTNFi_LRM\tnTNFi_MIX1\tnTNFi_MIX2\tpred_LRM\tpred_MIX1\tpred_MIX2\n", file=output_file,append=TRUE)

for (i in 1:NUM_CHEM_ID){
    if (i > 8){
    output_string <- ''
    chemID <- colnames(data_profile)[i]
    print (chemID)
    chem_profile <- data_profile[,i]
#     print (chem_profile)
        
    #mtx
    LRM <- lm(chem_profile ~ mtx , data = data_profile)
    mix_1 <- lmer(chem_profile ~ mtx + (1|patient_dummyID) , data = data_profile)
    mix_2 <- lmer(chem_profile ~ mtx + SEX + AGE + (1|patient_dummyID) , data = data_profile)    
    
    LRM_stats <- summary(LRM)$fstatistic
    LRM_sig_1 <- pf(LRM_stats[1], LRM_stats[2], LRM_stats[3], lower.tail=FALSE)
    mix_sig_1 <- anova(mix_1)$Pr[1]
#     print (anova(mix_1)$Pr[1])
#     print (mix_sig_1)
    mix_sig_2 <- anova(mix_2)$Pr[1]
    
    mtx_output_string <- paste(chemID, LRM_sig_1, mix_sig_1, mix_sig_2, sep="\t")
#     print (mtx_output_string)
    #csdmard
    LRM <- lm(chem_profile ~ csdmard , data = data_profile)
    LRM_stats <- summary(LRM)$fstatistic
    mix_1 <- lmer(chem_profile ~ csdmard + (1|patient_dummyID) , data = data_profile)
    mix_2 <- lmer(chem_profile ~ csdmard + SEX + AGE + (1|patient_dummyID) , data = data_profile)    
    
    LRM_sig_1 <- pf(LRM_stats[1], LRM_stats[2], LRM_stats[3], lower.tail=FALSE)
    mix_sig_1 <- anova(mix_1)$Pr[1]
    mix_sig_2 <- anova(mix_2)$Pr[1]       
        
    csdmard_output_string <- paste(LRM_sig_1, mix_sig_1, mix_sig_2, sep="\t")

    #tnfi
    LRM <- lm(chem_profile ~ tnfi , data = data_profile)
    LRM_stats <- summary(LRM)$fstatistic
    mix_1 <- lmer(chem_profile ~ tnfi + (1|patient_dummyID) , data = data_profile)
    mix_2 <- lmer(chem_profile ~ tnfi + SEX + AGE + (1|patient_dummyID) , data = data_profile)    
    
    LRM_sig_1 <- pf(LRM_stats[1], LRM_stats[2], LRM_stats[3], lower.tail=FALSE)
    mix_sig_1 <- anova(mix_1)$Pr[1]
    mix_sig_2 <- anova(mix_2)$Pr[1]       
        
    tnfi_output_string <- paste(LRM_sig_1, mix_sig_1, mix_sig_2, sep="\t")    
    
    #ntnfi
    LRM <- lm(chem_profile ~ ntnfi , data = data_profile)
    LRM_stats <- summary(LRM)$fstatistic
    mix_1 <- lmer(chem_profile ~ ntnfi + (1|patient_dummyID) , data = data_profile)
    mix_2 <- lmer(chem_profile ~ ntnfi + SEX + AGE + (1|patient_dummyID) , data = data_profile)    
    
    LRM_sig_1 <- pf(LRM_stats[1], LRM_stats[2], LRM_stats[3], lower.tail=FALSE)
    mix_sig_1 <- anova(mix_1)$Pr[1]
    mix_sig_2 <- anova(mix_2)$Pr[1]       
        
    ntnfi_output_string <- paste(LRM_sig_1, mix_sig_1, mix_sig_2, sep="\t")  
        
    #pred
    LRM <- lm(chem_profile ~ pred , data = data_profile)
    LRM_stats <- summary(LRM)$fstatistic
    mix_1 <- lmer(chem_profile ~ pred + (1|patient_dummyID) , data = data_profile)
    mix_2 <- lmer(chem_profile ~ pred + SEX + AGE + (1|patient_dummyID) , data = data_profile)    
    
    LRM_sig_1 <- pf(LRM_stats[1], LRM_stats[2], LRM_stats[3], lower.tail=FALSE)
    mix_sig_1 <- anova(mix_1)$Pr[1]
    mix_sig_2 <- anova(mix_2)$Pr[1]       
        
    pred_output_string <- paste(LRM_sig_1, mix_sig_1, mix_sig_2, sep="\t") 
    
    output_string <- paste(mtx_output_string, csdmard_output_string, tnfi_output_string, ntnfi_output_string, pred_output_string,"\n" , sep ="\t")
    cat(output_string, file=output_file,append=TRUE)
    }
}

print ("Done")

[1] "100002254"


boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular



[1] "100000257"
[1] "100001620"
[1] "565"
[1] "100020546"
[1] "100001254"
[1] "100004575"
[1] "100001092"
[1] "100003151"
[1] "803"
[1] "796"
[1] "100001266"
[1] "415"
[1] "100008933"
[1] "100020425"
[1] "503"
[1] "100020427"
[1] "1090"
[1] "100001051"
[1] "100000776"
[1] "100001253"
[1] "100001257"
[1] "100004088"
[1] "100009082"
[1] "1162"
[1] "100000096"
[1] "363"
[1] "100002945"
[1] "100002466"
[1] "100001577"
[1] "100001586"
[1] "407"
[1] "100021374"
[1] "100020204"
[1] "100020423"
[1] "100003397"
[1] "100002488"
[1] "100015838"
[1] "250"
[1] "100001391"
[1] "1442"
[1] "100003892"
[1] "100000042"
[1] "100015831"
[1] "100020414"
[1] "572"
[1] "338"
[1] "1087"


boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular



[1] "100000010"
[1] "100021141"
[1] "100001950"
[1] "100001635"
[1] "100015839"
[1] "100001197"
[1] "100000939"
[1] "100001313"
[1] "100000007"
[1] "100002911"
[1] "1489"
[1] "100020361"
[1] "100001104"
[1] "100000269"
[1] "100020866"
[1] "100001510"
[1] "100001767"
[1] "171"
[1] "344"
[1] "Done"
